#### avoid conflicts when more than one thread needs to access a single variable or other resource.
For example, consider a program that does some kind of processing, and keeps track of how many items it has processed:
```python
counter = 0

def process_item(item):
    global counter
    ... do something with item ...
    counter += 1
```

#### Atomic Operations 

The simplest way to synchronize access to shared variables or other resources is to rely on atomic operations in the interpreter. An atomic operation is an operation that is carried out in a single execution step, without any chance that another thread gets control.

In general, this approach only works if the shared resource consists of a single instance of a core data type, such as a string variable, a number, or a list or dictionary. Here are some thread-safe operations:

    reading or replacing a single instance attribute
    reading or replacing a single global variable
    fetching an item from a list
    modifying a list in place (e.g. adding an item using append)
    fetching an item from a dictionary
    modifying a dictionary in place (e.g. adding an item, or calling the clear method)

### Locks

```python
from threading import Lock
lock = Lock()

lock.acquire()
try:
    ... access shared resource
finally:
    lock.release() # release lock, no matter what
```

#### or..

```python
with lock:
    ... access shared resource
```

#### A possible problem with locks....

```python
lock = threading.Lock()

def get_first_part():
    lock.acquire()
    try:
        ... fetch data for first part from shared object
    finally:
        lock.release()
    return data

def get_second_part():
    lock.acquire()
    try:
        ... fetch data for second part from shared object
    finally:
        lock.release()
    return data
    

def get_both_parts():
    first = get_first_part() #call 1
    second = get_second_part() # call 2
    return first, second
    
```
#### But what if some other thread modifies the resource between call1 and call2 ?


#### So we need one more lock to tie both together
```python
def get_both_parts():
    lock.acquire()
    try:
        first = get_first_part()
        second = get_second_part()
    finally:
        lock.release()
    return first, second
 ```
 
 #### Now that is another problem! The individual access functions will get stuck..


### Re-Entrant Locks (RLock)
The RLock class is a version of simple locking that only blocks if the lock is held by **another** thread.

```python
lock = threading.Lock()
lock.acquire()
lock.acquire() # this will block

```

#### But with RLock

```python

lock = threading.RLock()
lock.acquire()
lock.acquire() # this won't block
```

#### Note that this lock keeps track of the recursion level, so you still need to call release once for each call to acquire.



## Semaphores

** A semaphore has an internal counter rather than a lock flag** 

** it only blocks if more than a given number of threads have attempted to hold the semaphore. **

** Depending on how the semaphore is initialized, this allows multiple threads to access the same code section simultaneously. **


```python
max_connections = 10

semaphore = threading.BoundedSemaphore(max_connections)
semaphore = threading.BoundedSemaphore()
semaphore.acquire() # decrements the counter
... access the shared resource
semaphore.release() # increments the counter

```

## Signalling

Although the point of using multiple threads is to spin separate operations off to run concurrently, there are times when it is important to be able to synchronize the operations in two or more threads. 

A simple way to communicate between threads is using **Event** objects. 

An Event manages an internal flag that callers can either set() or clear(). 

Other threads can wait() for the flag to be set(), effectively blocking progress until allowed to continue.

In [1]:
import logging
import threading
import time

logging.basicConfig(level=logging.DEBUG,
                    format='(%(threadName)-10s) %(message)s',
                    )
                    
def wait_for_event(e):
    """Wait for the event to be set before doing anything"""
    logging.debug('wait_for_event starting')
    event_is_set = e.wait()
    logging.debug('event set: %s', event_is_set)

def wait_for_event_timeout(e, t):
    """Wait t seconds and then timeout"""
    while not e.isSet():
        logging.debug('wait_for_event_timeout starting')
        event_is_set = e.wait(t)
        logging.debug('event set: %s', event_is_set)
        if event_is_set:
            logging.debug('processing event')
        else:
            logging.debug('doing other work')


e = threading.Event()
t1 = threading.Thread(name='block', 
                      target=wait_for_event,
                      args=(e,))
t1.start()

t2 = threading.Thread(name='non-block', 
                      target=wait_for_event_timeout, 
                      args=(e, 2))
t2.start()

logging.debug('Waiting before calling Event.set()')
time.sleep(3)
e.set()
logging.debug('Event is set')